### Agents

Agents in essence are LLMs that accomplish a specific task. They can be supplemented with tools, structured output, and handoff to other agents.

In [2]:
%pip install openai-agents python-dotenv -qU

Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
from dotenv import load_dotenv

load_dotenv()

api_key = os.environ.get("OPENAI_API_KEY")

if not api_key:
    raise ValueError("OPENAI_API_KEY is not set in the environment variables")

In [8]:
from agents import Agent, Runner

agent = Agent(
    name="Basic Agent",
    instructions="You are a helpful assistant. Respond on in all caps.",
    model="gpt-4o-mini"
)

result = await Runner.run(agent, "Whats up, how are you doing today?")
result.final_output

"I'M DOING GREAT, THANK YOU! HOW ABOUT YOU?"

In [7]:
joke_agent = Agent(
    name="Joke Agent",
    instructions="You are a joke teller. You are given a topic and you need to tell a joke about it.",
)

topic = "Manchester United"
result = await Runner.run(joke_agent, topic)
result.final_output

"Why don't Manchester United players play hide and seek?\n\nBecause good luck hiding when the entire city can see you haven't scored!"

In [10]:
language_agent = Agent(
    name="Language Agent",
    instructions="You are a language expert. You are given a joke and you need to rewrite it in a different language.",
)

joke_result = await Runner.run(joke_agent, topic)
translated_result = await Runner.run(language_agent, f"Translate this joke to French: {joke_result.final_output}")
print(f"Original joke:\n{joke_result.final_output}\n")
print(f"Translated joke:\n{translated_result.final_output}")

Original joke:
Why did the Manchester United fan bring a ladder to the game?

Because he heard the trophies were up for grabs!

Translated joke:
Pourquoi le supporter de Manchester United a-t-il apporté une échelle au match ?

Parce qu'il avait entendu dire que les trophées étaient à saisir !


### Structured Outputs

Structured outputs are a way to format the output of an LLM in a structured manner. This can be useful for tasks that require specific formatting or data extraction.

In [14]:
from pydantic import BaseModel
from agents import Agent

class Recipe(BaseModel):
    title: str
    ingredients: list[str]
    cooking_time: int # in minutes
    servings: int

recipe_agent = Agent(
    name="Recipe Agent", 
    instructions=("You are an agent for creating recipes. You will be given the name of a food and your job"
                  " is to output that as an actual detailed recipe. The cooking time should be in minutes."),
    output_type=Recipe
)

response = await Runner.run(recipe_agent, "Oatmeal with Almond Butter")
recipe = response.final_output
print(recipe)

title='Creamy Almond Butter Oatmeal' ingredients=['1 cup rolled oats', '2 cups almond milk or water', '2 tablespoons almond butter', '1 tablespoon honey or maple syrup', '1 teaspoon vanilla extract', '1/2 teaspoon cinnamon', 'Pinch of salt', 'Sliced almonds (optional, for topping)', 'Fresh berries (optional, for topping)'] cooking_time=10 servings=2


### Tool Calling

Tool calling is a way to extend the capabilities of an LLM by allowing it to call external tools or APIs. This can be useful for tasks that require access to external data or services.

In [15]:
from agents import Agent, function_tool

@function_tool
def get_weather(city: str) -> str:
    print(f"Getting weather for {city}")
    return "sunny"

@function_tool
def get_temperature(city: str) -> str:
    print(f"Getting temperature for {city}")
    return "70 degrees"

agent = Agent(
    name="Weather Agent",
    instructions="You are the local weather agent. You are given a city and you need to tell the weather and temperature. For any unrelated queries, say I cant help with that.",
    tools=[get_weather, get_temperature]
)

result = await Runner.run(agent, "Yerevan")
result.final_output

Getting weather for Yerevan
Getting temperature for Yerevan


'The weather in Yerevan is sunny with a temperature of 70 degrees Fahrenheit.'

In [16]:
from agents import WebSearchTool

transfer_market_agent = Agent(
    name="Arsenal Scout",
    instructions="You are a football transfer market agent. Your job is to find players for Arsenal FC to sign that play as a striker.",
    tools=[WebSearchTool()]
)

result = await Runner.run(transfer_market_agent, "find strikers")
print(result.final_output)

To find potential strikers for Arsenal FC, here are a few promising players who could be considered based on their talent and performance. Please note that the actual details might require more up-to-date scouting reports or discussions with clubs:

1. **Victor Osimhen** (Napoli)
   - Known for his speed, strength, and goal-scoring ability.
   
2. **Jonathan David** (LOSC Lille)
   - A versatile forward with a strong scoring record in Ligue 1.

3. **Darwin Núñez** (Liverpool)
   - Powerful striker known for his physical presence and finishing.

4. **Alexander Isak** (Newcastle United)
   - Tall, technical, and capable of scoring goals in different ways.

5. **Dusan Vlahovic** (Juventus)
   - Strong forward with impressive goal-scoring capabilities.

These players have shown potential and could fit into Arsenal's system. For the latest details on their availability, contract status, and transfer market interest, further scouting and discussions would be needed.
